# Blackhole emission models

`synthesizer` can utilise several different black hole emission models, i.e. a model that translates a set of physical properties to one or several spectral energy distributions. 

In [ ]:
from synthesizer.blackholes import UnifiedAGN
import matplotlib.pyplot as plt
import numpy as np
from unyt import deg

### `Template` model

Our simplest model is to just provide a template, either directly or from a file. This is simply scaled by the blackhole bolometric luminosity to produce the spectra.

### `UnifiedAGN` model

Our default model is the `UnifiedAGN` model. This combines a disc model with a model for the NLR, BLR, and torus.

This model is initialised with a `disc_model` (by default we use the model of Kubota and Done 2019  (https://ui.adsabs.harvard.edu/abs/2018MNRAS.480.1247K/abstract) and any additional fixed parameters.

In [ ]:
grid_dir = '../../../tests/test_grid/'
emission_model = UnifiedAGN(disc_model='test_grid_agn', photoionisation_model='', grid_dir=grid_dir)

There are various parameter lists associated with this model, e.g.

In [ ]:
emission_model.disc_parameters

The parameters of the grid(s):

In [ ]:
emission_model.grid_parameters

All parameters:

In [ ]:
emission_model.parameters

Parameters fixed at initialisation:

In [ ]:
emission_model.fixed_parameters

Variable parameters which are need to be provided by the call to `get_spectra` etc.

In [ ]:
emission_model.variable_parameters

For the UnifiedAGN model a handful of spectra can be created (and returned) if desired:

In [ ]:
emission_model.available_spectra

#### Generating spectra

We can generate spectra using the get_spectra command and passing the necessary parameters. In this case the blackhole mass, the accretion rate expressed as a fraction of the Eddington rate (`accretion_rate_eddington`), and the cosine of the inclination (`cosine_inclination`).  Other disc models have different parameters.

In [ ]:
parameters_dict, spectra = emission_model.get_spectra(mass=1E9, accretion_rate_eddington=1., cosine_inclination=0.5, metallicity=0.01)

Note, the combined spectra from the AGN is labelled as "intrinsic". While moderately confusing this is to align with the labelling of stellar population spectra. When associated with a galaxy the "emergent", "dust", and "total" spectra can be generated if diffuse dust attenuation (and emission) is specified.

In [ ]:
for id in ['disc_incident', 'disc', 'blr', 'nlr', 'torus', 'intrinsic']:

    sed = spectra[id]

    plt.plot(np.log10(sed.lam), np.log10(sed.luminosity), label = id)

plt.xlim([2,6])
plt.ylim(np.max(np.log10(sed.luminosity))-4, np.max(np.log10(sed.luminosity))+0.2)
plt.xlabel(r'$\log_{10}(\lambda/\AA)$')
plt.ylabel(r'$\log_{10}(\nu L_{\nu})$')
plt.legend()
plt.show()

#### Inclination dependence

When the inclination is high ($>90^{\circ}-\theta_{torus}$) the disc and BLR are assumed to be entirely obscured by the dusty torus. In this case the emission from the disc and BLR is set to zero.

In addition some disc models (e.g. AGNSED) have a dependence on the inclination. This effects the emission observed from the disc but not how the NLR, BLR, and torus are illuminated.

In [ ]:
for inclination in [0, 15, 30, 45, 60, 75, 90]:
    cosine_inclination = np.cos((inclination*deg).to_value('rad'))
    parameters_dict, spectra = emission_model.get_spectra(mass=1E9, accretion_rate_eddington=1., cosine_inclination=cosine_inclination, metallicity=0.01)
    
    sed = spectra['intrinsic']
    plt.plot(np.log10(sed.lam), np.log10(sed.luminosity), label = inclination)

plt.xlim([2,6])
plt.ylim(np.max(np.log10(sed.luminosity))-4, np.max(np.log10(sed.luminosity))+0.2)
plt.xlabel(r'$\log_{10}(\lambda/\AA)$')
plt.ylabel(r'$\log_{10}(\nu L_{\nu})$')
plt.legend()
plt.show()